In [1]:
import os
import networkx as nx
import matplotlib.pyplot as plt
import cython
import numpy as np
import pandas as pd
import datetime
import pylab

In [6]:
# load in merged data
data_path = '../../../UCB_dept_merge_CorrectDeptLabels.csv'
merged = pd.read_csv(data_path)

In [7]:
#clean up the department dataset
def convert_strings_to_specials(s):
    s = s.replace(' ', '_')
    s = s.replace(':', '_')
    s = s.replace('#', 'num')
    s = s.lower()
    return s
merged.columns = [convert_strings_to_specials(col) for col in merged.columns]

In [55]:
#convert creation_date to datetime object
merged['creation_date']=pd.to_datetime(pd.Series(merged['creation_date']))

unnamed__0                           int64
po_id                                int64
po_num                              object
creation_date               datetime64[ns]
supplier_name                       object
item_type                           object
product_description                 object
manufacturer                        object
quantity                             int64
unit_price                         float64
department                         float64
buyer__first_name                   object
buyer__last_name                    object
po_closed_date                      object
department_name                     object
spend                              float64
department_number                   object
department_name_stripped            object
department_name_update              object
date                               float64
dtype: object

In [56]:
merged['creation_date'] = merged['creation_date'].str.replace('00:00:00', '')
merged.dtypes

unnamed__0                           int64
po_id                                int64
po_num                              object
creation_date               datetime64[ns]
supplier_name                       object
item_type                           object
product_description                 object
manufacturer                        object
quantity                             int64
unit_price                         float64
department                         float64
buyer__first_name                   object
buyer__last_name                    object
po_closed_date                      object
department_name                     object
spend                              float64
department_number                   object
department_name_stripped            object
department_name_update              object
date                               float64
dtype: object

In [57]:
#create seperate year, month, day colums

merged['year'] = merged['creation_date'].apply(lambda x: x[:4])
merged['month'] = merged['creation_date'].apply(lambda x: x[5:7])
merged['day'] = merged['creation_date'].apply(lambda x: x[8:10])

TypeError: 'NaTType' object has no attribute '__getitem__'

In [12]:
firstDate = min(merged['creation_date'])
lastDate = max(merged['creation_date'])
timeframe = lastDate-firstDate
subsection = timeframe/12

In [13]:
firstDate

Timestamp('2012-01-18 00:00:00')

In [15]:
merged.head(20)

,unnamed__0,po_id,po_num,creation_date,supplier_name,item_type,product_description,manufacturer,quantity,unit_price,department,buyer__first_name,buyer__last_name,po_closed_date,department_name,spend,department_number,department_name_stripped,department_name_update
0,0,29847876,BB00195887,2013-05-31,GIVE SOMETHING BACK,SQ Hosted Product,"PORTABLE COMBINATION LAPTOP LOCK, 6 FT. CARBON...",KENSINGTON,30,24.32,NaN,Dustin,Miller,2013-07-16 00:00:00,UIAPA UB Academic Year,795.26,UIAPA,UB Academic Year,NaN
1,1,29847864,BB00195886,2013-05-31,GRAINGER INC,PunchOut Product,"Wall Mount Fan, Oscillating, Number of Speeds ...",AIR KING,1,35.58,NaN,Erin,Pinkston,2013-08-08 00:00:00,UKHDS Unit 1 Apt Admin,38.78,UKHDS,Unit 1 Apt Admin,Housing & Dining Services
2,2,29847796,BB00195884,2013-05-31,BELLCO GLASS INC,NonCatalog Product,"Septum Stopper, 20mm Blue Butyl Rubber QtyPerC...",NaN,1,77.41,NaN,William,Wolf,NaN,CCHEM RES Research,104.67,CCHEM,RES Research,Dept Of Chemistry
3,3,29847820,BB00195885,2013-05-31,CHEMGLASS LIFE SCIENCES LLC,NonCatalog Product,"Column, Chromatography, 24/40 Outer Joint, 100...",NaN,1,108.00,NaN,William,Wolf,2014-10-30 00:00:00,CCHEM RES Research,71.77,CCHEM,RES Research,Dept Of Chemistry
4,4,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,"Bottles, Media/Lab; Wheaton;Graduated; With ru...",Wheaton Science Products Inc,1,135.38,NaN,Donald C.,RIO,NaN,IMMCB BH Research,317.31,IMMCB,BH Research,Molecular & Cell Biology
5,5,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,"Flasks, Cell Culture; Corning; Wide neck; Stac...",Corning Life Sciences Plastic,1,81.29,NaN,Donald C.,RIO,NaN,IMMCB BH Research,317.31,IMMCB,BH Research,Molecular & Cell Biology
6,6,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,Filter unit; EMD Millipore; Steriflip; Disposa...,Emd Millipore Corp,1,96.56,NaN,Donald C.,RIO,NaN,IMMCB BH Research,317.31,IMMCB,BH Research,Molecular & Cell Biology
7,7,29847493,BB00195879,2013-05-31,EPPENDORF NORTH AMERICA INC,NonCatalog Product,Deepwell plate 96/2000 µl - Regular package-st...,NaN,2,150.00,NaN,Brandi,Bankston,2013-08-12 00:00:00,IMMCB LSA Research,263.20,IMMCB,LSA Research,Molecular & Cell Biology
8,8,29847492,BB00195878,2013-05-31,VWR INTERNATIONAL INC,SQ Hosted Product,CALCIUM CHLORIDE DIHYD 500GM Calcium Chloride ...,AMRESCO INC,2,29.85,NaN,Melissa,Sanchez,2013-08-08 00:00:00,IMMCB LSA Research,65.07,IMMCB,LSA Research,Molecular & Cell Biology
9,9,29847443,BB00195872,2013-05-31,EMD MILLIPORE CORPORATION (Formerly EMD Chemic...,SQ Hosted Product,"Cancer Hydrolase KIAA1363 Inhibitor, JW480",Calbiochem/EMD Millipore,1,94.00,NaN,Lindsay Shayna,Roberts,NaN,MDNST NS RESEARCH GRANTS,120.36,MDNST,NS RESEARCH GRANTS,Nutritional Sci & Tox Dept


In [27]:
#iterator for getting month,year
def month_year_iter( start_month, start_year, end_month, end_year ):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range( ym_start, ym_end ):
        y, m = divmod( ym, 12 )
        yield y, m+1

In [39]:
[(1,2,3),1,2]+[(1,2,3),1,2]

[(1, 2, 3), 1, 2, (1, 2, 3), 1, 2]

In [41]:
#function to give a bitartite graph and centrality measure of department and supplier by time
def testCentralityGraph(data, set1, set2, time_column):
    results = []
    def calculate(set1, set2, current):
        G = nx.graph()
        G.add_nodes_from(set1,bipartite = 0)
        G.add_nodes_From(set2,bipartite = 1)
        edgeList = [tuple(x) for x in current.values]
        G.add_edges_from(edgeList)
        pos = nx.networkx.spring_layout(G)
        nx.draw_networkx_nodes(G, pos, nodelist = nodelistDept, node_color = 'r', node_size = 50)
        nx.draw_networkx_nodes(G, pos, nodelist = nodelistSup, node_color= 'w', node_size=50)
        #draw edges
        nx.draw_networkx_edges(G,pos,width=0.5,alpha=0.5)
        nx.draw_networkx_edges(G,pos,edgelist=edgeList)
        bottom_nodes, top_nodes = nx.bipartite.sets(G)
        projected_bottom = nx.bipartite.projected_graph(G, bottom_nodes, multigraph = True)
        projected_top = nx.bipartite.projected_graph(G, top_nodes, multigraph = True)
        top_density, bottom_density = nx.bipartite.density(G, top_nodes), nx.bipartite.density(G, bottom_nodes)
        top_degree = nx.degree_centrality(projected_top)
        bot_degree = nx.degree_centrality(projected_bottom)
        top_between = nx.betweenness_centrality(projected_top)
        bot_between = nx.betweenness_centrality(projected_bottom)
        top_projected_density = nx.density(projected_top)
        bot_projected_density = nx.density(projected_bottom)
        
        return (G,projected_top,projected_bottom,top_density,bottom_density,top_degree,bot_degree,top_between,bot_between,
                top_projected_density,bot_projected_density)

        
            
    subset = data[[set1,set2,time_column]]
    firstDate = min(data[time_column])
    lastDate = max(data[time_column])
    iter = month_year_iter(firstDate.month, firstDate.year, lastDate.month, lastDate.year)
    for month,year in iter:
        current = subset[subset[time_column].month == month and subset[time_column].year == year]
        current = current[[set1,set2]]
        current.dropna()
        s1 = current[set1]
        s2 = current[set2]
        temp = calculate(s1,s2,current)
        results +=[(temp,month,year)]

    return results

In [42]:
masterlist = testCentralityGraph(merged,'department_name_update', 'supplier_name','creation_date')

AttributeError: 'Series' object has no attribute 'month'

[            department_name_update                      supplier_name
271042     Construction and Design               Walters & Wolf Glass
271043     Construction and Design                       DLR ELECTRIC
271044     Construction and Design                         RSANALYSIS
271045     Construction and Design                  THE GORDIAN GROUP
271046                   Economics                         OFFICE MAX
271047           Coll of Chem Dean                  FISHER SCIENTIFIC
271048           Coll of Chem Dean                  FISHER SCIENTIFIC
271049           Coll of Chem Dean                  FISHER SCIENTIFIC
271050           Coll of Chem Dean                  FISHER SCIENTIFIC
271051           Coll of Chem Dean                  FISHER SCIENTIFIC
271052           Coll of Chem Dean                  FISHER SCIENTIFIC
271053           BA Bancroft Admin                    MINUTEMAN PRESS
271054           BA Bancroft Admin                    MINUTEMAN PRESS
271055           BA

In [56]:
#subset by date
first3Months = deptSupplier[deptSupplier['creation_date'] < datetime.datetime(2012,3,1)]
first3Months = first3Months.dropna() 
len(first3Months)

636

In [57]:
#keep only the unique values
wanted = first3Months[['department_name','supplier_name']]
wanted.drop_duplicates()

,department_name,supplier_name
273362,MDNST,SANTA CRUZ BIOTECHNOLOGY INC
273361,MDNST,OPEN BIOSYSTEMS INC
273360,MEPMB,FISHER SCIENTIFIC
273354,MDNST,FISHER SCIENTIFIC
273341,ALPDC,GLS ARCHITECT LANDSCAPE ARCHITECT
273342,EERES,VWR INTERNATIONAL INC
273348,ALPDC,VILA CONSTRUCTION COMPANY
273349,MEPMB,APPLE COMPUTER INC
273351,ALPDC,Carl Zeiss Microscopy LLC
273332,EERES,ALKO OFFICE SUPPLY


In [58]:
#make a graph
test= nx.Graph()
edges = [tuple(x) for x in wanted.values]
test.add_edges_from(edges)
#position = {department:0 for department in wanted["department_name"]}
#for supplier in wanted["supplier_name"]: position[supplier] = 1
values = range(0,len(test.nodes()))
#len(position) ==len(values)

In [ ]:
nx.draw(test, node_color = values,with_labels = True)
plt.show()
plt.savefig("test.png")